<a href="https://colab.research.google.com/github/Lucas-Kuo/SVHN_yolov5/blob/master/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cloning the repo

In [ ]:
!git clone https://github.com/Lucas-Kuo/SVHN_yolov5.git
%cd SVHN_yolov5

Import and download necessary packets

In [ ]:
import gdown
import os
import shutil
import json
!pip install PyYAML==6.0

Download testing dataset from Google Drive and unzip it

In [ ]:
url = "https://drive.google.com/u/0/uc?id=1Fm-avdeNgzhPxhvia0iw9yZzcoOggy7I&export=download"
output = "test.zip"
gdown.download(url, output, quiet=False)

In [ ]:
!mkdir test
!unzip test.zip -d test

Download the trained model from Google Drive

In [ ]:
url = "https://drive.google.com/u/0/uc?id=1J386qflqdWkA_bqJ5dkRardHUpv14APT&export=download" # s_16
output = "best.pt"
gdown.download(url, output, quiet=False)

Clear the directory we're going to store our result

(It's highly recommended that you execute this cell everytime before running inference.py)

In [ ]:
if os.path.exists("runs/detect"):
  shutil.rmtree("runs/detect")

Running inference

(This may take several minutes since I'm aiming for high mAP and set the confidence threshold quite low)

P.S. Inference speed time will also be shown in the console of this cell

In [ ]:
# Inference 
!python detect.py --weights best.pt --source test/test/ --device 0 --save-txt --save-conf --iou-thres 0.6  --conf-thres 0.0001 --nosave

After the cell above has finished executing, we're going to collect the results and produce answer.json

In [ ]:
answer_dirs = os.listdir("runs/detect/exp/labels")
ANSWER_BASE_DIR = "runs/detect/exp/labels/"
result_to_json = []

answer_dirs.sort(key = lambda x: int(x[:-4]))

for answer_dir in answer_dirs:
  # parse image id from "xxxxx.txt"
  img_id = answer_dir[:-4]
  
  with open(ANSWER_BASE_DIR+answer_dir, "r") as f:
    lines = f.readlines()
    for line in lines:
      det_box_info = {}
      # line contains: {class} {x}{y}{w}{h} {confidence}
      args = line.split()

      det_box_info["image_id"] = int(img_id)
      det_box_info["category_id"] = int(args[0])
      det_box_info["bbox"] = [float(args[1]), float(args[2]), float(args[3]), float(args[4])]
      det_box_info["score"] = float(args[5])

      result_to_json.append(det_box_info)
  
json_object = json.dumps(result_to_json, indent=4)

with open("answer.json", "w") as outfile:
  outfile.write(json_object)


Now, answer.json is created in /content/SVHN_yolov5